In [1]:
#Import modules
import os
import numpy
import psycopg2
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

In [2]:
data_path = '/Users/amanda/Documents/Projects/insight/data/raw/onet-skills-db/'

#Load ONet Data
df_Skills = pd.read_excel(os.path.join(data_path,'Skills.xlsx'))
df_AltTitles = pd.read_excel(os.path.join(data_path,'Alternate Titles.xlsx'))
df_WorkAct = pd.read_excel(os.path.join(data_path,'Work Activities.xlsx'))
df_DWARef = pd.read_excel(os.path.join(data_path,'DWA Reference.xlsx'))
df_IWA = pd.read_excel(os.path.join(data_path,'DWA Reference.xlsx'))

In [4]:
#Build matrix for analysis
jobsList = df_Skills['Title'].unique()
skillsList = df_Skills['Element Name'].unique()

#Make a matrix for the scores
skillsMat = numpy.zeros((len(jobsList),len(skillsList)))
for i in range(len(jobsList)):
    for j in range(len(skillsList)):
        skillsMat[i,j] = df_Skills[(df_Skills['Title']==jobsList[i]) & (df_Skills['Element Name']==skillsList[j]) & (df_Skills['Scale ID']=='IM')]['Data Value']
        
#Make a matrix of the scores        
actsList = df_WorkAct['Element Name'].unique()
actsMat = numpy.zeros((len(jobsList),len(actsList)))
outList = []
for i in range(len(jobsList)):
    for j in range(len(actsList)):
        actsMat[i,j] = df_WorkAct[(df_WorkAct['Title']==jobsList[i]) & (df_WorkAct['Element Name']==actsList[j]) & (df_WorkAct['Scale ID']=='IM')]['Data Value']

In [5]:
#Create dataframe with desired info
df_write_skills = pd.DataFrame(skillsMat,columns=skillsList)
df_write_skills['title'] = jobsList
df_write_acts = pd.DataFrame(actsMat,columns=actsList)
df_write_acts['title'] = jobsList

#Write to database
#Create or load database
dbname = 'onet-skills-db'
username = 'amanda' # change this to your username
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

#Write to tables
df_write_skills.to_sql('skills_table',engine,if_exists='replace')
df_write_acts.to_sql('acts_table',engine,if_exists='replace')
df_IWA.to_sql('iwa_table',engine,if_exists='replace')
df_DWARef.to_sql('dwa_table',engine,if_exists='replace')
df_AltTitles[['O*NET-SOC Code','Title','Alternate Title']].to_sql('alttitles_table',engine,if_exists='replace')

postgres://amanda@localhost/onet-skills-db
True
